# HDS5210-2022 Fall Midterm

In the midterm exercise, you're going to use all the programming and data management skills you've developed so far to build a pricing calculator that will calculate how much money should be reimbursed for the visits in a CSV file. To do this, you will need to get allowed amounts (aka rates) from a JSON file, apply some special rules, and then calculate various totals by hospital or by month.

Each step of the midterm will build up to form your final solution. Along the way, I've provided plenty of test cases to make sure you're getting each step correct.

All functions require docstrings with a description and at least one test case.

The midterm is due Monday, October 24th at 11:59 PM CST.

---

## Step 1: Average Rate

With the `/data/negotiated_rates.json` file as input for your first function, read all the `allowed_amount` attributes and calculate an average allowed amount over all rates in the file.

Your function should be named **average_rate()**, take the file's name as it's input parameter, and return a float as the result.

**ROUND YOUR ANSWER TO 2 DECIMAL PLACES**

In [1]:
import json
def average_rate(file):
    """
    This function reads all the 'allowed_amout' attributes and calculates an average allowed amount over all rates in the file. 
    This fucntion takes the file's name as it's input parameter, and return a float as the result.
    
    >>> average_rate('/data/negotiated_rates.json')
    38.67
    
    """
    jason_file = json.load(open(file))
    total = 0
    count=0
    avg=0
    out_of_network = jason_file.get('out_of_network')
    for value in out_of_network:
        allowed_amounts = value.get('allowed_amounts')
        for amount in allowed_amounts:
            payments = amount.get('payments')
            total+=payments['allowed_amount']
            count +=1
            avg = round((total/count),2)
    return avg



In [2]:
assert(average_rate('/data/negotiated_rates.json') == 38.67)

In [3]:
average_rate('/data/negotiated_rates.json')

38.67

In [4]:
import doctest
doctest.run_docstring_examples(average_rate, globals(), verbose=True)

Finding tests in NoName
Trying:
    average_rate('/data/negotiated_rates.json')
Expecting:
    38.67
ok


---

## Step 2: Rate for a Billing Code / Service Code Combination

For the next step, we need to be able to look up the allowed amount for any given billing code / service code combination.

In this data, the billing code represents the service or procedure that was provided. The service code represents the type of site where the service was provided.

Your function should be named **get_rate()** and take three parameters: the JSON file name, the billing code, and the service code. If your code can't find that combination in the file, it should return None.

In [5]:
import json
def get_rate(file, billing_code1, service_code1):
    """
    This function take three parameters: the JSON file name, the billing code, and the service code, and returns 
    the the allowed amount for any given billing code / service code combination.
    
    >>> get_rate('/data/negotiated_rates.json','G0283','11')
    8.78
    
    >>> get_rate('/data/negotiated_rates.json','97140','11')
    20.03
    """
    jason_file = json.load(open(file))
    dic = {}
    out_of_network = jason_file.get('out_of_network')
    for index, value in enumerate(out_of_network):
        billing_code = out_of_network[index]['billing_code']
        allowed_amounts = value.get('allowed_amounts')
        for amount in allowed_amounts:
            service_code = amount.get('service_code')
            payments = amount.get('payments')['allowed_amount']
            if billing_code not in dic:
                dic[billing_code] = []
            dic[billing_code].append([service_code,payments])
    data = dic.get(billing_code1) #data contains the list of service code and payments for the given key
    len_data = len(data) 
    i = 0
    the_allowed_amount= 0 
    while i < len_data:
        if data[i][0]==service_code1:
            the_allowed_amount = data[i][1]
        i+=1
        
    
    return the_allowed_amount
    

In [6]:
assert (get_rate('/data/negotiated_rates.json','G0283','11') == 8.78)
assert (get_rate('/data/negotiated_rates.json','97140','11') == 20.03)
assert (get_rate('/data/negotiated_rates.json','97110','12') == 26.62)

In [7]:
doctest.run_docstring_examples(get_rate, globals(), verbose=True)

Finding tests in NoName
Trying:
    get_rate('/data/negotiated_rates.json','G0283','11')
Expecting:
    8.78
ok
Trying:
    get_rate('/data/negotiated_rates.json','97140','11')
Expecting:
    20.03
ok


In [8]:
get_rate('/data/negotiated_rates.json','97112','11')

32.9

---

## Part 3: Special Rules for Rates

There are some special rules for adjusting rates depending on a patient's age and the day of week.  (These are made up. Not from the real world.) In the next part of the midterm, you'll need to create a function that applies these extra rules to a rate and return the adjusted value.

1. If the day of week is Monday, charge only 75% of the allowed amount.
2. If the patient's age is 65 or higher, charge only 50% of the allowed amount.
3. If's both Monday and the patient's age is 65 or higher, charge only 50% of the allowed amount.
4. If neither of these conditions are true, charge the whole amount.

Your function should be named **get_adjusted_rate()** and take five parameters: file name, billing code, service code, patient age, and visit date.  Your function should return the adjusted rate (based on the rules above) or None if the rate couldn't be found in the file.

Note that your function will take a date in the form `%Y-%m-%d` ([see datetime.strptime()](https://docs.python.org/3/library/datetime.html#datetime.datetime.strptime)) and will need to calculate the [day of week](https://docs.python.org/3/library/datetime.html#datetime.date.weekday).

**ROUND YOUR ANSWER TO 2 DECIMAL PLACES**

In [9]:
from datetime import datetime

def get_adjusted_rate(file, billing_code, service_code, pat_age, visit_date):
    """
    This function take five parameters: file name, billing code, service code, patient age, and visit date, and return
    the adjusted rate (based on the rules below) or None if the rate couldn't be found in the file.
        1. If the day of week is Monday, charge only 75% of the allowed amount.
        2. If the patient's age is 65 or higher, charge only 50% of the allowed amount.
        3. If's both Monday and the patient's age is 65 or higher, charge only 50% of the allowed amount.
        4. If neither of these conditions are true, charge the whole amount.
    
    >>> get_adjusted_rate('/data/negotiated_rates.json','G0283','11',22,'2022-01-07')
    8.78
    
    >>> get_adjusted_rate('/data/negotiated_rates.json','G0283','11',76,'2022-01-08')
    4.39
    
    
    """
    allowed_amount = get_rate(file, billing_code, service_code)
    date = datetime.strptime(visit_date, '%Y-%m-%d')
    #Return the day of the week as an integer, where Monday is 0 and Sunday is 6.
    day_of_week = date.weekday()
    if day_of_week == 0 and pat_age>=65:
        allowed_amount = round(allowed_amount*0.50,2)
    elif day_of_week ==0:
        allowed_amount = round(allowed_amount*0.75,2)
    elif pat_age>=65:
        allowed_amount = round(allowed_amount*0.50,2)
    else:
        allowed_amount
        
        
    return allowed_amount
    

In [10]:
assert (get_adjusted_rate('/data/negotiated_rates.json','G0283','11',22,'2022-01-07') == 8.78)
assert (get_adjusted_rate('/data/negotiated_rates.json','G0283','11',22,'2022-01-03') == 6.58)
assert (get_adjusted_rate('/data/negotiated_rates.json','G0283','11',76,'2022-01-03') == 4.39)
assert (get_adjusted_rate('/data/negotiated_rates.json','G0283','11',76,'2022-01-08') == 4.39)

In [11]:
doctest.run_docstring_examples(get_adjusted_rate, globals(), verbose=True)

Finding tests in NoName
Trying:
    get_adjusted_rate('/data/negotiated_rates.json','G0283','11',22,'2022-01-07')
Expecting:
    8.78
ok
Trying:
    get_adjusted_rate('/data/negotiated_rates.json','G0283','11',76,'2022-01-08')
Expecting:
    4.39
ok


---

## Step 4: Calculate total payments for a list of visits

Last step, we're going to put your get_adjusted_rate() function to work on a list of visits from a few hospitals in the St. Louis region.

Your last function, **summarize_reimbursement()** needs to read an encounter file and the negotiated rates file, compute an adjusted rate for each encounter (row) in the input file, and return two dictionaries of information:
1. Total expected reimbursement by month
2. Total expected reimbursement by hospital

Your should be able to run your function as `by_month, by_hospital = summarize_reimbursement(visits, negotiated_rates)` and have the two answers below:

**by_month**
```json
{
    '2021-05': 192.38,
    '2021-03': 378.72,
    '2021-07': 277.67,
    '2021-06': 236.53,
    '2021-11': 229.7,
    '2021-10': 234.52,
    '2021-12': 297.87,
    '2021-04': 337.7,
    '2021-09': 160.4,
    '2021-01': 111.91,
    '2021-02': 158.55,
    '2021-08': 152.28
}
```

**by_hospital**
```json
{
    'Missouri Baptist': 514.18,
    'SSM DePaul': 460.02,
    'SLU Hospital': 409.67,
    'Barnes Jewish': 485.7,
    'Mercy Springfield': 518.59,
    'Mercy St. Louis': 380.07
}
```

**Round your totals to 2 decimal places**

**If rates are not found, just ignore them**

In [12]:
import json
import csv
from datetime import datetime
import csv

def summarize_reimbursement(file, file1):
    """
    This function uses 'visits' and 'negotiated_rates' data and returns two dictionaries that shows
    total expected reimbursement by month and total expected reimbursement by hospital
    
    >>> summarize_reimbursement('/data/visits.csv','/data/negotiated_rates.json')[0]['2021-07']
    277.67
    
    >>> summarize_reimbursement('/data/visits.csv','/data/negotiated_rates.json')[0]['2021-03']
    378.72
    
    
    """
    result_hospital = {}
    result_month={}
    total_byHospital1=0
    total_byHospital2=0
    total_byHospital3=0
    total_byHospital4=0
    total_byHospital5=0
    total_byHospital6=0
    total_byMonth1=0
    total_byMonth2=0
    total_byMonth3=0
    total_byMonth4=0
    total_byMonth5=0
    total_byMonth6=0
    total_byMonth7=0
    total_byMonth8=0
    total_byMonth9=0
    total_byMonth10=0
    total_byMonth11=0
    total_byMonth12=0
    
    h_list = ['Missouri Baptist','SSM DePaul','SLU Hospital','Barnes Jewish','Mercy Springfield','Mercy St. Louis']

    with open(file) as f:
        reader = csv.reader(f, delimiter=',')
        for row in reader:
            (SYSTEM, HOSPITAL, PATIENT_ID, PATIENT_AGE, BILLING_CODE, SERVICE_CODE, VISIT_DATE) = row
            if  HOSPITAL == 'Missouri Baptist':
                PATIENT_AGE = int(PATIENT_AGE)
                amount=get_adjusted_rate(file1, BILLING_CODE, SERVICE_CODE, PATIENT_AGE, VISIT_DATE)
                total_byHospital1+= amount
                result_hospital[HOSPITAL] = round(total_byHospital1,2)
            if  HOSPITAL == 'SSM DePaul':
                PATIENT_AGE = int(PATIENT_AGE)
                amount2=get_adjusted_rate(file1, BILLING_CODE, SERVICE_CODE, PATIENT_AGE, VISIT_DATE)
                total_byHospital2+= amount2
                result_hospital[HOSPITAL] = round(total_byHospital2,2)
            if  HOSPITAL == 'SLU Hospital':
                PATIENT_AGE = int(PATIENT_AGE)
                amount3=get_adjusted_rate(file1, BILLING_CODE, SERVICE_CODE, PATIENT_AGE, VISIT_DATE)
                total_byHospital3+= amount3
                result_hospital[HOSPITAL] = round(total_byHospital3,2)
            if  HOSPITAL == 'Barnes Jewish':
                PATIENT_AGE = int(PATIENT_AGE)
                amount4=get_adjusted_rate(file1, BILLING_CODE, SERVICE_CODE, PATIENT_AGE, VISIT_DATE)
                total_byHospital4+= amount4
                result_hospital[HOSPITAL] = round(total_byHospital4,2)
            if  HOSPITAL == 'Mercy Springfield':
                PATIENT_AGE = int(PATIENT_AGE)
                amount5=get_adjusted_rate(file1, BILLING_CODE, SERVICE_CODE, PATIENT_AGE, VISIT_DATE)
                total_byHospital5+= amount5
                result_hospital[HOSPITAL] = round(total_byHospital5,2)
            if  HOSPITAL == 'Mercy St. Louis':
                PATIENT_AGE = int(PATIENT_AGE)
                amount6=get_adjusted_rate(file1, BILLING_CODE, SERVICE_CODE, PATIENT_AGE, VISIT_DATE)
                total_byHospital6+= amount6
                result_hospital[HOSPITAL] = round(total_byHospital6,2)
                
                
                
            if  VISIT_DATE[:-3] == '2021-05':
                PATIENT_AGE = int(PATIENT_AGE)
                amount7=get_adjusted_rate(file1, BILLING_CODE, SERVICE_CODE, PATIENT_AGE, VISIT_DATE)
                total_byMonth1+= amount7
                result_month[VISIT_DATE[:-3]] = round(total_byMonth1,2)
            if  VISIT_DATE[:-3] == '2021-03':
                PATIENT_AGE = int(PATIENT_AGE)
                amount8=get_adjusted_rate(file1, BILLING_CODE, SERVICE_CODE, PATIENT_AGE, VISIT_DATE)
                total_byMonth2+= amount8
                result_month[VISIT_DATE[:-3]] = round(total_byMonth2,2)
            if  VISIT_DATE[:-3] == '2021-07':
                PATIENT_AGE = int(PATIENT_AGE)
                amount9=get_adjusted_rate(file1, BILLING_CODE, SERVICE_CODE, PATIENT_AGE, VISIT_DATE)
                total_byMonth3+= amount9
                result_month[VISIT_DATE[:-3]] = round(total_byMonth3,2)
            if  VISIT_DATE[:-3] == '2021-06':
                PATIENT_AGE = int(PATIENT_AGE)
                amount10=get_adjusted_rate(file1, BILLING_CODE, SERVICE_CODE, PATIENT_AGE, VISIT_DATE)
                total_byMonth4+= amount10
                result_month[VISIT_DATE[:-3]] = round(total_byMonth4,2)
            if  VISIT_DATE[:-3] == '2021-11':
                PATIENT_AGE = int(PATIENT_AGE)
                amount11=get_adjusted_rate(file1, BILLING_CODE, SERVICE_CODE, PATIENT_AGE, VISIT_DATE)
                total_byMonth5+= amount11
                result_month[VISIT_DATE[:-3]] = round(total_byMonth5,2)
            if  VISIT_DATE[:-3] == '2021-10':
                PATIENT_AGE = int(PATIENT_AGE)
                amount12=get_adjusted_rate(file1, BILLING_CODE, SERVICE_CODE, PATIENT_AGE, VISIT_DATE)
                total_byMonth6+= amount12
                result_month[VISIT_DATE[:-3]] = round(total_byMonth6,2)
            if  VISIT_DATE[:-3] == '2021-12':
                PATIENT_AGE = int(PATIENT_AGE)
                amount13=get_adjusted_rate(file1, BILLING_CODE, SERVICE_CODE, PATIENT_AGE, VISIT_DATE)
                total_byMonth7+= amount13
                result_month[VISIT_DATE[:-3]] = round(total_byMonth7,2)
            if  VISIT_DATE[:-3] == '2021-04':
                PATIENT_AGE = int(PATIENT_AGE)
                amount14=get_adjusted_rate(file1, BILLING_CODE, SERVICE_CODE, PATIENT_AGE, VISIT_DATE)
                total_byMonth8+= amount14
                result_month[VISIT_DATE[:-3]] = round(total_byMonth8,2)
            if  VISIT_DATE[:-3] == '2021-09':
                PATIENT_AGE = int(PATIENT_AGE)
                amount15=get_adjusted_rate(file1, BILLING_CODE, SERVICE_CODE, PATIENT_AGE, VISIT_DATE)
                total_byMonth9+= amount15
                result_month[VISIT_DATE[:-3]] = round(total_byMonth9,2)
            if  VISIT_DATE[:-3] == '2021-01':
                PATIENT_AGE = int(PATIENT_AGE)
                amount16=get_adjusted_rate(file1, BILLING_CODE, SERVICE_CODE, PATIENT_AGE, VISIT_DATE)
                total_byMonth10+= amount16
                result_month[VISIT_DATE[:-3]] = round(total_byMonth10,2)
            if  VISIT_DATE[:-3] == '2021-02':
                PATIENT_AGE = int(PATIENT_AGE)
                amount17=get_adjusted_rate(file1, BILLING_CODE, SERVICE_CODE, PATIENT_AGE, VISIT_DATE)
                total_byMonth11+= amount17
                result_month[VISIT_DATE[:-3]] = round(total_byMonth11,2)
            if  VISIT_DATE[:-3] == '2021-08':
                PATIENT_AGE = int(PATIENT_AGE)
                amount18=get_adjusted_rate(file1, BILLING_CODE, SERVICE_CODE, PATIENT_AGE, VISIT_DATE)
                total_byMonth12+= amount18
                result_month[VISIT_DATE[:-3]] = round(total_byMonth12,2)
        
          
            
             
                
            
    return result_month, result_hospital

In [13]:
summarize_reimbursement('/data/visits.csv','/data/negotiated_rates.json')

({'2021-05': 192.38,
  '2021-03': 378.72,
  '2021-07': 277.67,
  '2021-06': 236.53,
  '2021-11': 229.7,
  '2021-10': 234.52,
  '2021-12': 297.87,
  '2021-04': 337.7,
  '2021-09': 160.4,
  '2021-02': 158.55,
  '2021-01': 111.91,
  '2021-08': 152.28},
 {'Missouri Baptist': 514.18,
  'SSM DePaul': 460.02,
  'SLU Hospital': 409.67,
  'Mercy St. Louis': 380.07,
  'Barnes Jewish': 485.7,
  'Mercy Springfield': 518.59})

In [14]:
assert (summarize_reimbursement('/data/visits.csv','/data/negotiated_rates.json')[0]['2021-07'] == 277.67)
assert (summarize_reimbursement('/data/visits.csv','/data/negotiated_rates.json')[0]['2021-03'] == 378.72)
assert (summarize_reimbursement('/data/visits.csv','/data/negotiated_rates.json')[1]['Mercy St. Louis'] == 380.07)
assert (summarize_reimbursement('/data/visits.csv','/data/negotiated_rates.json')[1]['Mercy Springfield'] == 518.59)

In [15]:
summarize_reimbursement('/data/visits.csv','/data/negotiated_rates.json')[0]['2021-07']

277.67

In [16]:
doctest.run_docstring_examples(summarize_reimbursement, globals(), verbose=True)

Finding tests in NoName
Trying:
    summarize_reimbursement('/data/visits.csv','/data/negotiated_rates.json')[0]['2021-07']
Expecting:
    277.67
ok
Trying:
    summarize_reimbursement('/data/visits.csv','/data/negotiated_rates.json')[0]['2021-03']
Expecting:
    378.72
ok
